In [40]:
import json
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

import pandas as pd
import seaborn as sns

c.execute('''SELECT * FROM Matches WHERE season = 2011;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df.head()

992


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [41]:
class weather_prepare:
    def __init__(self,df):
        self.df = df

    # prepare date for use in dark sky api -> convert to unix time
    def prepare_dataframe_for_weather(self):
        
        df.set_index('Match_ID', inplace = True)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Unix_Date'] = (df['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [42]:
weather_prepare.prepare_dataframe_for_weather(df)

In [43]:
df.head()

,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unix_Date
Match_ID,,,,,,,,,
1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1333152000
1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,1323561600
1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,1313193600
1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,1322352000
1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,1329523200


In [63]:
date_test = {'dates':['1310688000','1336262400','1336867200']}

In [62]:
date_test['dates'][0]

'1310688000'

In [66]:
from apikey import *
class weather_scrape:
    
#     date = date_test
    
    def __init__(self,df,dates):
        self.df = df
        self.url = 'https://api.darksky.net/forecast'
        self.key = API_Key
        self.latitude = "52.5200"
        self.longitude = "13.4050"
        self.exclude = 'currently,flags,minutely,hourly,alerts'
        self.dates = dates
    
    
    def api_call(self,date):
    
        good_url = "{}/{}/{},{},{}?exclude={}".format(self.url,self.key,self.latitude, self.longitude, self.date, self.exclude)
        
        response = requests.get(good_url)
        print(response)
        weather = response.json()['daily']['data'][0]['icon']
        return weather


    def all_the_weather(self,dates):
        weather_dict = {}
        for date in dates:
            weather_data = api_call(self,dates)
            weather_dict.update({date: weather_data})
        return weather_dict
            

#     def adding_weather_to_df(self):
#         weather_dict = self.all_the_weather(date)
#         wdf = pd.DataFrame(weather_dict, index = ['weather'])
#         wdf2 = pd.DataFrame.transpose(wdf)
#         wdf2 = wdf2.reset_index()
#         wdf2.rename(columns={'index': 'Unix_Date'}, inplace=True)
#         df = pd.merge(df, wdf2, how = 'left', on = 'Unix_Date')
#         return df.head()
    
    ##maybe put adding weather outside the class??

In [ ]:
def adding_weather_to_df(dates):
        weather_dict = weather_scrape.all_the_weather(dates)
        wdf = pd.DataFrame(weather_dict, index = ['weather'])
        wdf2 = pd.DataFrame.transpose(wdf)
        wdf2 = wdf2.reset_index()
        wdf2.rename(columns={'index': 'Unix_Date'}, inplace=True)
        df = pd.merge(df, wdf2, how = 'left', on = 'Unix_Date')
        return df.head()

In [69]:
weather_scrape.api_call(date_test)

AttributeError: 'dict' object has no attribute 'url'

Match_ID
1092    1333152000
1093    1323561600
1094    1313193600
1095    1322352000
1096    1329523200
Name: Unix_Date, dtype: int64